In [78]:
import json
from collections import Counter
from PIL import ImageColor
from scipy.spatial import KDTree
from webcolors import (
    CSS3_HEX_TO_NAMES,
    hex_to_rgb,
)

with open('merged.json') as p:
    data = json.load(p)

In [ ]:
css3_db = CSS3_HEX_TO_NAMES

# Function to get the color name from an RGB value
def convert_rgb_to_names(rgb_tuple):
    
    # a dictionary of all the hex and their respective names in css3
    names = []
    rgb_values = []    
    for color_hex, color_name in css3_db.items():
        names.append(color_name)
        rgb_values.append(hex_to_rgb(color_hex))
    
    kdt_db = KDTree(rgb_values)    
    distance, index = kdt_db.query(rgb_tuple)
    return names[index]

In [139]:
def remake_pokemon_json(data, use_main_color = False):
    new_dict = []
    for dex_num, poke in data.items():
        new_poke_dict = dict()
        color_names = set()

        for color in poke['colors']:
            color_rgb = ImageColor.getcolor(color, "RGB")
            color_name = convert_rgb_to_names(color_rgb)

            color_names.add(color_name)


        new_poke_dict['id'] = dex_num

        try:
            new_poke_dict['name'] = poke['name']
        except:
            new_poke_dict['name'] = ''

        new_poke_dict['types'] = poke['types']

        try:
            new_poke_dict['main_color'] = poke['color']
        except:
            new_poke_dict['main_color'] = ''

        if (use_main_color and new_poke_dict['main_color'] != ''):           
            color_names.add(new_poke_dict['main_color'])

        new_poke_dict['colors'] = list(color_names)
        new_poke_dict['colors'].sort()

        new_dict.append(new_poke_dict)

    return new_dict

In [140]:
new_poke_dict = remake_pokemon_json(data, use_main_color=True)

In [141]:
print(new_poke_dict[0])
print(new_poke_dict[110])
print(new_poke_dict[270])

{'id': '001', 'name': 'bulbasaur', 'types': ['Grass', 'Poison'], 'main_color': 'green', 'colors': ['black', 'darkseagreen', 'gray', 'green', 'white']}
{'id': '111', 'name': 'rhyhorn', 'types': ['Ground', 'Rock'], 'main_color': 'gray', 'colors': ['black', 'darkgray', 'darkslategray', 'dimgray', 'gray', 'white']}
{'id': '271', 'name': 'lombre', 'types': ['Water', 'Grass'], 'main_color': 'green', 'colors': ['black', 'darkkhaki', 'green', 'olivedrab', 'silver', 'white']}


In [136]:
hex_codes = []

for poke in new_poke_dict:
    color_list = poke['colors']
    hex_codes.extend(color_list)

hex_codes_count = Counter(hex_codes)

hex_codes_count.most_common(10)

[('white', 790),
 ('black', 605),
 ('darkslategray', 388),
 ('dimgray', 284),
 ('gray', 245),
 ('blue', 157),
 ('silver', 157),
 ('darkgray', 157),
 ('brown', 145),
 ('indianred', 112)]

In [98]:
typings = []

for poke in new_poke_dict:
    if len(poke['types']) > 1:
        type_tuple = (poke['types'][0], poke['types'][1])
    else:
        type_tuple = (poke['types'][0],)
    
    typings.append(type_tuple)

typings_count = Counter(typings)

typings_count.most_common(10)

[(('Normal',), 69),
 (('Water',), 67),
 (('Grass',), 43),
 (('Psychic',), 37),
 (('Fire',), 33),
 (('Electric',), 32),
 (('Fighting',), 27),
 (('Normal', 'Flying'), 26),
 (('Bug',), 19),
 (('Fairy',), 18)]

In [114]:
import networkx as nx

def create_pokemon_graph(data, key):

    #--- Your code starts here
    graph = nx.Graph()

    nodes = []
    edges = []
    for pokemon in data:
        for node in pokemon[key]:
            nodes.append(node)
            for item in pokemon[key]:
                edges.append((item, node))

    graph.add_nodes_from(nodes)
    graph.add_edges_from(edges)

    return graph

In [115]:
pokemon_color_graph = create_pokemon_graph(new_poke_dict, 'colors')

for edge in pokemon_color_graph.edges:
    if (edge[0] == 'black'):
        print(edge)

('black', 'black')
('black', 'darkseagreen')
('black', 'gray')
('black', 'white')
('black', 'mediumaquamarine')
('black', 'palevioletred')
('black', 'slategray')
('black', 'cadetblue')
('black', 'seagreen')
('black', 'snow')
('black', 'bisque')
('black', 'burlywood')
('black', 'rosybrown')
('black', 'chocolate')
('black', 'salmon')
('black', 'wheat')
('black', 'skyblue')
('black', 'tan')
('black', 'dimgray')
('black', 'silver')
('black', 'khaki')
('black', 'peru')
('black', 'darkkhaki')
('black', 'olivedrab')
('black', 'yellowgreen')
('black', 'ghostwhite')
('black', 'indianred')
('black', 'lightgray')
('black', 'darkolivegreen')
('black', 'gainsboro')
('black', 'sandybrown')
('black', 'sienna')
('black', 'palegoldenrod')
('black', 'coral')
('black', 'darkgray')
('black', 'darksalmon')
('black', 'plum')
('black', 'saddlebrown')
('black', 'lightsteelblue')
('black', 'teal')
('black', 'mistyrose')
('black', 'antiquewhite')
('black', 'pink')
('black', 'forestgreen')
('black', 'steelblue')

In [109]:
color_type_combos = []

skip_white = False

for poke in new_poke_dict:
    for color in poke['colors']:
        if len(poke['types']) > 1:
            type_tuple = (poke['types'][0], poke['types'][1])
        else:
            type_tuple = (poke['types'][0],)

        if (color != 'white' or not skip_white):
            combo_tuple = (type_tuple, color)
            color_type_combos.append(combo_tuple)

combo_counter = Counter(color_type_combos)

combo_counter.most_common(50)

[((('Normal',), 'white'), 62),
 ((('Normal',), 'black'), 53),
 ((('Water',), 'white'), 50),
 ((('Water',), 'black'), 44),
 ((('Grass',), 'white'), 37),
 ((('Psychic',), 'white'), 36),
 ((('Grass',), 'black'), 35),
 ((('Fire',), 'white'), 31),
 ((('Water',), 'darkslategray'), 26),
 ((('Electric',), 'white'), 25),
 ((('Fighting',), 'white'), 25),
 ((('Psychic',), 'black'), 25),
 ((('Normal', 'Flying'), 'white'), 23),
 ((('Normal',), 'dimgray'), 23),
 ((('Normal',), 'darkslategray'), 21),
 ((('Fire',), 'black'), 20),
 ((('Normal',), 'gray'), 20),
 ((('Normal', 'Flying'), 'darkslategray'), 20),
 ((('Fighting',), 'darkslategray'), 18),
 ((('Normal',), 'rosybrown'), 18),
 ((('Normal',), 'darkgray'), 17),
 ((('Electric',), 'black'), 17),
 ((('Electric',), 'darkslategray'), 17),
 ((('Water',), 'dimgray'), 16),
 ((('Poison',), 'white'), 16),
 ((('Fire',), 'darkslategray'), 16),
 ((('Grass',), 'dimgray'), 16),
 ((('Grass',), 'darkseagreen'), 16),
 ((('Water',), 'skyblue'), 15),
 ((('Water',), 's